# TM10007 Group Assignment Machine Learning
#### Sara Arman, Judith Essenburg, George Franssen, Naomi Verkerk

## Google colab environment

In [65]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/naomiverkerk/TM10007.git

## Import


In [1]:
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition

from load_data import load_data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


## Loading Data

In [2]:
# Moeten hier nu wel even checken of X niet bevat in welke groep hij behoort
data = load_data() 
data = data.replace('#DIV/0!', np.nan)
X = data
Y = data['label']

# Dit betekent dat ik nu label eruit heb gehaald voor input zodat missing values werkt
# En ook even andere 2 categorical variables zodat het scalen werkt
X.pop('label')

print(f'The number of samples/patients: {len(data.index)}')
print(f'The number of columns/features: {len(data.columns)}')


The number of samples/patients: 167
The number of columns/features: 724


## Splitting in train and test data

In [3]:
# the code to split, after that we inspect the data

# Misschien hier nog stratify = Y gebruiken? Zag ik in voorbeeld
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size = 0.2, random_state = 4, stratify = Y)


## Checking/exploring the dataset

In [4]:
#data.dtypes == 'object'
# Seperate numerical variables and categorical variables
#num_vars = data.columns[data.dtypes != 'object']
cat_vars = data.columns[data.dtypes == 'object']

#print(len(num_vars))
#print(len(cat_vars))
print(cat_vars)
# to check which features have most missing values
#missing_values = data[num_vars].isnull().sum().sort_values(ascending=False)

#print(missing_values)
#print(f'The total number of features with 1 or more missing values is {missing_values_multiple}')

Index([], dtype='object')


## Features weghalen die te veel missing values hebben

In [5]:
#Je bepaalt de ratio --> heb de helft genomen en je dropt degene die boven threshold zitten
acceptabele_ratio = 0.5
train_size = len(X_train.index)
removal_rate = round(train_size*0.5)

X_train = X_train.dropna(axis=1, thresh=removal_rate)

# maar dit moet je vervolgens ook nog droppen bij je X_test
common_cols = list(set(X_train.columns).intersection(X_test.columns))
X_test = X_test[common_cols]



## Imputation --> Missing values

In [11]:
# In order to do scaling and PCA, first the missing values need to be filled in. 
# With scaling, it appeared that some values are infinite, so that is why those will be removed with a large finite number

X_train_missing_median = X_train.fillna(X_train.median())
X_train_missing_median = np.nan_to_num(X_train_missing_median)
X_test_missing_median = X_test.fillna(X_test.median())
X_test_missing_median = np.nan_to_num(X_test_missing_median)




## Scaling

In [14]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train_missing_median)
X_train_scaled = scaler.transform(X_train_missing_median)
X_train_scaled = np.nan_to_num(X_train_scaled)
X_test_scaled = scaler.transform(X_test_missing_median)
X_test_scaled = np.nan_to_num(X_test_scaled)


C:\Users\nwver\miniconda3\lib\site-packages\sklearn\utils\extmath.py:1014: RuntimeWarning: overflow encountered in square
  temp **= 2
C:\Users\nwver\miniconda3\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction ** 2 / new_sample_count
C:\Users\nwver\miniconda3\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction ** 2 / new_sample_count
C:\Users\nwver\miniconda3\lib\site-packages\sklearn\preprocessing\_data.py:80: RuntimeWarning: overflow encountered in multiply
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


## Perform a PCA

In [15]:
pca = decomposition.PCA(n_components=2)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').